## Importing required libraries

In [1]:
# suppressing warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import random
import math
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Iterable, List

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k

from torchdata.datapipes.iter import IterableWrapper, Mapper

from nltk.translate.bleu_score import sentence_bleu

!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------ --------------------------------- 2.1/12.8 MB 11.8 MB/s eta 0:00:01
     ----------- ---------------------------- 3.7/12.8 MB 12.1 MB/s eta 0:00:01
     ------------------ --------------------- 5.8/12.8 MB 10.7 MB/s eta 0:00:01
     ------------------------ --------------- 7.9/12.8 MB 9.6 MB/s eta 0:00:01
     ------------------------------- -------- 10.0/12.8 MB 9.6 MB/s eta 0:00:01
     -------------------------------------- - 12.3/12.8 MB 9.9 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 9.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ----- ---------------------------------- 2.1/14.6 MB 13.0 MB/s eta 0:00:01
     ------------- -------------------------- 5.0/14.6 MB 13.7 MB/s eta 0:00:01
     -------------------- --------

## Checking if CUDA is available

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


## The Encoder

In [4]:
# a rrn implemented using "nn.EmbeddingBag", "nn.LSTM", "nn.Dropout" functions from "pytorch" library
class Encoder(nn.Module):
    def __init__(self, vocab_len, emb_dim, hid_dim, n_layers, dropout_prob):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(vocab_len, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, input_batch):
        embed = self.dropout(self.embedding(input_batch))
        embed = embed.to(device)
        outputs, (hidden, cell) = self.lstm(embed)

        return hidden, cell

#### Encoder -> example of one forward pass

In [5]:
# dummy data
vocab_len = 8
emb_dim = 10
hid_dim = 8
n_layers = 1
dropout_prob = 0.5

# instantiating the model
encoder_t = Encoder(vocab_len, emb_dim, hid_dim, n_layers, dropout_prob).to(device)

In [32]:
# dummy data 
src_batch = torch.tensor([0,3,4,2,1]) #where 0,3,4,2,1 are vocab indecies
src_batch = src_batch.t().to(device)

# getting the embedding of the text token indices
embedded = encoder_t.embedding(src_batch)
# appling dropout to embedded
embedded_dropout = encoder_t.dropout(embedded)
embedded_dropout.to(device)
# passing through the lstm
outputs, (hidden, cell) = encoder_t.lstm(embedded_dropout)

print(f"Input(src) tensor  [shape -> {src_batch.shape[0]}]:-\n", src_batch)
print(f"\nEmbedded tokens  [shape -> {embedded.shape[0]}]:-\n", embedded)
print(f"\nAfter dropout:-  [shape -> {embedded_dropout.shape[0]}]\n", embedded_dropout)
print(f"\nHidden:-         [shape -> {hidden.shape[0]}]\n", hidden)
print("\nCell:-           \n", cell)
print("\nOutput:-         \n", outputs)

Input(src) tensor  [shape - 5]:-
 tensor([0, 3, 4, 2, 1], device='cuda:0')

Embedded tokens  [shape - 5]:-
 tensor([[-0.7081,  0.8048, -1.4479,  0.5327, -0.9940,  1.1202, -1.5410,  0.4600,
          0.1129, -1.0188],
        [-0.3430, -0.9896, -2.9051, -1.0750,  0.6061, -0.0841,  1.0588, -0.8620,
          0.6973,  1.1518],
        [-0.7842,  0.5488,  0.0891, -1.9937,  0.5474,  0.6157, -0.6369,  1.3035,
          1.6213,  0.1466],
        [-0.3078, -1.6557, -0.0123, -0.8786, -0.1814, -1.0452,  0.3461, -1.3314,
         -0.6607, -0.6228],
        [ 0.1651,  0.2855, -0.7564,  1.4578, -0.9304, -1.7696, -0.4859, -0.9556,
         -0.1792, -0.2691]], device='cuda:0', grad_fn=<EmbeddingBackward0>)

After dropout:-  [shape - 5]
 tensor([[-0.0000,  1.6097, -0.0000,  1.0653, -1.9880,  0.0000, -0.0000,  0.9201,
          0.2257, -0.0000],
        [-0.6859, -0.0000, -0.0000, -0.0000,  1.2123, -0.1682,  0.0000, -1.7239,
          1.3947,  2.3035],
        [-1.5684,  0.0000,  0.0000, -3.9874,  0.00

In [8]:
src_batch

tensor([0, 3, 4, 2, 1])